# **Importación de librerías** 
Se importan las librerías necesarias para el análisis
- *requests*:  para hacer solicitudes HTTP.
- *pandas*: para la manipulación de datos.
- *numpy*: para operaciones numéricas.
- *matplotlib y seaborn*: para la visualización de datos.

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

from scipy import stats
from sklearn.preprocessing import StandardScaler

import data_util as util

import warnings

# Ignorar todas las advertencias
warnings.filterwarnings("ignore")

# **Carga de datos**
En esta fase se cargarán los datos preprocesados anteriormente, sin datos faltantes ni outliers.

In [ ]:
# Cargar csv
ruta = "../minidatathonF5_emma/data/covid19_all.csv"
df = pd.read_csv(ruta, sep=',')
df.head(2)

# Cantidad registros importados
df.shape

In [ ]:
df.columns

* Convertir las variables al formato adecuado para realizar las visualizaciones

In [ ]:
# Convertir la columna "date" en un objeto de fecha y hora
df['date'] = pd.to_datetime(df['date'])

# Transformar la columna 'state' a categorías
df['state'] = df['state'].astype('category')
df.info()

# **Análisis Exploratorio Univariante**

El análisis univariante es una técnica estadística que se utiliza para analizar y describir una variable individual en un conjunto de datos. En lugar de examinar las relaciones entre múltiples variables, el análisis univariante se enfoca en comprender las características y propiedades de una sola variable a la vez.

El análisis univariante puede incluir varias técnicas y medidas, como:

1. Medidas de tendencia central: Estas medidas, como la media, la mediana y la moda, se utilizan para describir el valor central o típico de una variable.

2. Medidas de dispersión: Estas medidas, como la desviación estándar y el rango, se utilizan para describir cuánto se dispersan los valores de una variable alrededor de la medida central.

3. Gráficos de distribución: Los histogramas, diagramas de caja y bigotes, y gráficos de densidad son ejemplos de gráficos utilizados en el análisis univariante para visualizar la distribución de una variable y detectar patrones o formas específicas.

4. Análisis de frecuencia: Este análisis implica contar y resumir la frecuencia de diferentes valores o categorías de una variable. Se puede utilizar para identificar valores atípicos, valores más comunes o patrones de frecuencia.

5. Pruebas de hipótesis: En el análisis univariante, también se pueden realizar pruebas estadísticas para evaluar si hay diferencias significativas entre grupos o categorías de una variable.

El análisis univariante es una etapa importante en cualquier análisis de datos, ya que proporciona una comprensión inicial y detallada de una variable individual. Ayuda a resumir y describir las características clave de una variable, identificar patrones o tendencias y detectar valores atípicos o anómalos.

* Estadísticas Descriptivas

In [ ]:
# Se eliminan las columnas númericas que no interesan para las estadisticas 
df_num = df.drop(['date','state'],axis=1)
df_num.head(2)

In [ ]:
df_num.describe().round(2)

    Conclusiones Generales:
    Variabilidad Significativa: Hay una gran variabilidad en todas las variables, especialmente en los casos positivos, negativos y resultados de pruebas. Esto puede reflejar diferencias en la magnitud de la pandemia en diferentes regiones o en diferentes momentos.
    Tasas de Mortalidad y Hospitalización: Las tasas de mortalidad y hospitalización tienen una variabilidad moderada, pero en general parecen estar bajo control, con una tasa de mortalidad promedio del 2.32% y una tasa de hospitalización del 8.14%.
    Casos Extremos: Existen observaciones con valores extremos, como casos positivos y negativos muy altos, o incluso cero, lo cual sugiere que en algunas áreas o periodos la pandemia tuvo un impacto drásticamente diferente.
    Esta información puede ser útil para analizar tendencias y disparidades en el impacto de la pandemia, así como para orientar futuras políticas de salud.








* Visualización con Gráficos de Densidad

In [ ]:
numeric_columns = df_num.select_dtypes(include=np.number).columns.to_list()
num_plots = len(numeric_columns)
num_cols = 4
num_rows = (num_plots + num_cols - 1) // num_cols

fig, axs = plt.subplots(num_rows, num_cols, figsize=(12, 6))


for i, column in enumerate(numeric_columns):
    row = i // num_cols
    col = i % num_cols

    axs[row, col].set_title(f'{column}')
    sns.kdeplot(data=df, x=column, ax=axs[row, col])
    
# Eliminar subparcelas vacías si es necesario
if num_plots % num_cols != 0:
    fig.delaxes(axs[-1, -1])
    
# Agregar título general a la figura
plt.suptitle('GRÁFICOS DE DENSIDAD', fontsize=16)

# Ajustar el espacio entre el título general y los subplots
plt.subplots_adjust(top=1)
plt.tight_layout()
plt.show()

    Conclusiones Generales:
    Sesgo a la Derecha en Varias Variables: Variables como casos positivos, negativos, hospitalizaciones, muertes y resultados de pruebas muestran un sesgo a la derecha, lo que sugiere que la mayoría de las observaciones están en los valores más bajos, con algunas observaciones en valores mucho más altos.
    Distribuciones Bimodales: Las distribuciones de los casos negativos y la tasa de mortalidad parecen ser bimodales, lo que indica que existen dos subgrupos dentro de los datos, posiblemente reflejando diferencias entre regiones o períodos de tiempo.
    Colas Largas: En varias distribuciones, especialmente en casos positivos, muertes y resultados de pruebas, se observan colas largas hacia valores altos, indicando la presencia de valores extremos que podrían ser importantes para un análisis más detallado.
    Estos patrones pueden ayudar a identificar tendencias, anomalías y diferencias entre distintos grupos o periodos de tiempo en los datos, lo que podría ser útil para guiar la toma de decisiones y análisis más profundos.

* Histogramas para cada varible numérica en subplots

In [ ]:
# Graficar los histogramas en subplots
df_num.hist(bins=30, alpha=0.6, color='b', edgecolor='k', linewidth=1.5, figsize=(15, 10), layout=(3, 4))

# Ajustar el espaciado entre los subplots
plt.tight_layout()

# Agregar título general
plt.suptitle('Histograma por Variable', fontsize=16)

# Ajustar el espacio entre el título general y los subplots
plt.subplots_adjust(top=0.9)

# Mostrar los histogramas
plt.show()

    Conclusiones Generales:
    Sesgo a la Derecha en la Mayoría de las Variables: La mayoría de las variables, como casos positivos, negativos, hospitalizaciones, muertes y resultados de pruebas, muestran distribuciones sesgadas a la derecha, lo que sugiere que la mayoría de las observaciones tienen valores bajos, pero hay algunas con valores muy altos.
    Distribuciones Bimodales: Las distribuciones de los casos negativos y la tasa de mortalidad muestran una posible bimodalidad, lo que sugiere la presencia de subgrupos en los datos que podrían reflejar diferencias geográficas, temporales o demográficas.
    Variabilidad en la Tasa de Mortalidad y Hospitalización: Aunque hay concentraciones alrededor de ciertos valores, estas tasas también muestran variabilidad significativa entre las observaciones, lo que sugiere diferencias en la respuesta a la pandemia o en la gravedad de los casos en distintas regiones o periodos de tiempo.
    En resumen, los histogramas reflejan patrones similares a los gráficos de densidad, confirmando la concentración de datos en los valores bajos para la mayoría de las variables, pero también mostrando la existencia de subgrupos o valores extremos que podrían ser de interés para un análisis más profundo.

# **Visualización histórico de Fecha**

* Mediante está visualización se puede observar si existe estacionalidad en las 'Tasa de Mortalidad' y 'Tasa de Hospitalización' en el 
período (01/2020 a 03/2021)

In [ ]:
date_columns = df.select_dtypes(include='datetime64').columns.to_list()

for column in date_columns:
    plt.figure(figsize=(15,7))
    
    sns.lineplot(data=df.sample(1000), x=column, y='mortality_rate')
    plt.title('Tasa de Mortalidad')
    plt.xlim(pd.Timestamp('2020-01-01'), pd.Timestamp('2021-03-31'))
    plt.show()

    Conclusión General:
    El gráfico muestra una alta volatilidad en la tasa de mortalidad a lo largo del tiempo, con picos que reflejan momentos críticos de la pandemia y una tendencia general a la baja hacia 2021, posiblemente indicando mejoras en la respuesta global ante la pandemia. Sin embargo, las fluctuaciones constantes subrayan la complejidad de la situación y la variabilidad en la capacidad de respuesta en diferentes periodos y regiones.

In [ ]:

# Agrupar por semana para análisis de tendencias
df['week'] = df['date'].dt.to_period('W')
df_weekly = df.groupby(['state', 'week']).agg({
    'positive': 'sum',
    'death': 'sum',
    'hospitalized': 'sum'
}).reset_index()

In [ ]:
date_columns = df.select_dtypes(include='datetime64').columns.to_list()

for column in date_columns:
    plt.figure(figsize=(15,7))
    
    sns.lineplot(data=df.sample(1000), x=column, y='hospitalization_rate')
    plt.title('Tasa de Hospitalización')
    plt.xlim(pd.Timestamp('2020-01-01'), pd.Timestamp('2021-03-31'))
    plt.show()

    La tasa de hospitalización durante la pandemia de COVID-19 mostró una alta variabilidad con múltiples picos a lo largo de 2020, lo que refleja las fluctuaciones en la severidad de la pandemia y posibles cambios en las políticas de salud o capacidad hospitalaria. Aunque hubo un ligero descenso hacia finales de 2020, la tasa no presentó una tendencia clara y estuvo marcada por una considerable incertidumbre. Esto sugiere que la situación fue altamente dinámica y sujeta a múltiples factores contextuales a lo largo del año.

In [ ]:
# Asegurarnos de que la columna 'date' es de tipo datetime
df['date'] = pd.to_datetime(df['date'])

# Crear columna 'week' basada en la fecha
df['week'] = df['date'].dt.to_period('W')

In [ ]:
# Agrupar por semana para análisis de tendencias
df['week'] = df['date'].dt.to_period('W')
df_weekly = df.groupby(['state', 'week']).agg({
    'positive': 'sum',
    'death': 'sum',
    'hospitalized': 'sum'
}).reset_index()

# ANÁLISIS BIVARIANTE

El análisis bivariante se utiliza para examinar la relación o asociación entre dos variables en un conjunto de datos.

El análisis bivariante puede ser útil para:

1. Identificar correlaciones: El análisis bivariante permite determinar si existe una relación entre dos variables y si esa relación es positiva, negativa o no existe. Esto puede ayudar a comprender cómo una variable afecta a la otra y a identificar patrones o tendencias.

2. Realizar pruebas de hipótesis: El análisis bivariante puede usarse para evaluar si hay diferencias significativas entre dos grupos o categorías en función de una variable. Esto puede ayudar a determinar si una variable tiene un impacto estadísticamente significativo en otra variable.

3. Visualizar la relación: El análisis bivariante a menudo se representa mediante gráficos, como gráficos de dispersión, diagramas de cajas y bigotes, o gráficos de barras. Estos gráficos permiten visualizar la relación entre las dos variables y pueden revelar patrones o tendencias visuales.

4. Predecir valores: El análisis bivariante puede ayudar a predecir valores de una variable en función de otra variable. 

####  **Análisis de distribuciones**

* Casos positivos por Estado

In [ ]:
# Agrupamos por estado y sumamos los casos positivos
df_state = df.groupby('state').agg({
    'positive': 'sum'
}).reset_index()

# Gráfico de barras para mostrar los casos positivos acumulados por estado usando Seaborn y Matplotlib
plt.figure(figsize=(15, 8))
sns.set_palette('coolwarm')

# Ordenamos los estados por el número de casos positivos acumulados
df_state_sorted = df_state.sort_values(by='positive', ascending=False)

# Creamos el gráfico de barras
sns.barplot(x='state', y='positive', data=df_state_sorted)

plt.title('Casos positivos por estado')
plt.xlabel('Estado')
plt.ylabel('Casos positivos')
plt.xticks(rotation=45)
plt.show()

* Tasa de Mortalidad por Estado

In [ ]:
# Configurar el estilo de seaborn
sns.set_style("whitegrid")

# Crear la figura y los ejes
fig, ax = plt.subplots(figsize=(12, 6))

# Crear un boxenplot usando Seaborn
sns.boxenplot(data=df, x='state', y='mortality_rate',  palette='coolwarm')

# Rotar las etiquetas del eje x para mejorar la legibilidad
plt.xticks(rotation=90)

# Agregar título y etiquetas a los ejes
ax.set_title('Distribución de tasas de mortalidad por estado', fontsize=16, pad=20)
ax.set_title('Estado', fontsize=12)
ax.set_title('Tasa de mortalidad (%)', fontsize=12)

# Quitar las líneas de la cuadrícula
ax.grid(False)

# Ajustar los márgenes
plt.tight_layout()

# Mostrar el gráfico
plt.show()

#### **Visualización de Tendencias temporales**

* Casos semanales positivos a nivel nacional

In [ ]:
# Agrupamos por semana para obtener el total de casos positivos a nivel nacional
national_weekly = df_weekly.groupby('week')['positive'].sum().reset_index()

# Convertimos la columna 'week' a un formato adecuado para Seaborn (puede ser string o datetime)
national_weekly['week'] = national_weekly['week'].astype(str)

# Gráfico de líneas usando Seaborn
plt.figure(figsize=(15, 7))
sns.lineplot(data=national_weekly, x='week', y='positive')

plt.title('Casos positivos semanales de COVID-19 en EE.UU.')
plt.xlabel('Semana')
plt.ylabel('Casos positivos')
plt.xticks(rotation=45)

# Generar nuevas etiquetas "semana1", "semana2", ...
new_labels = [f'{i+1}ª week' for i in range(len(national_weekly))]

# Quitar las líneas de división
plt.grid(False)

# Aplicar las nuevas etiquetas al eje x 
plt.xticks(ticks=range(len(new_labels)), labels=new_labels, rotation=45)
plt.show()

* Tasa de Crecimiento diario en los Estados más afectados

In [ ]:
# Configurar el estilo de seaborn
sns.set_style("whitegrid")
sns.set_palette("deep")

# Calcular tasa de crecimiento diario
df['growth_rate'] = df.groupby('state')['positive'].pct_change() * 100

# Convertir la columna 'date' a datetime si aún no lo está
df['date'] = pd.to_datetime(df['date'])

# Filtrar datos hasta noviembre de 2020
df_filtered = df[df['date'] <= '2020-11-30']

# Obtener los 5 estados más afectados
top_5_states = df_filtered.groupby('state')['positive'].max().nlargest(5).index

# Crear la figura y los ejes
fig, ax = plt.subplots(figsize=(15, 8))

# Gráfico de líneas de tasa de crecimiento para los 5 estados más afectados
for state in top_5_states:
    state_data = df_filtered[df_filtered['state'] == state]
    sns.lineplot(x='date', y='growth_rate', data=state_data, label=state, ax=ax)

# Personalizar el gráfico
ax.set_title('Tasa de crecimiento diario en los 5 estados más afectados', fontsize=16, pad=20)
ax.set_xlabel('Fecha', fontsize=12)
ax.set_ylabel('Tasa de crecimiento (%)', fontsize=12)

# Quitar las líneas de la cuadrícula
ax.grid(False)

# Ajustar la leyenda
ax.legend(title='Estado', title_fontsize='12', fontsize='10', loc='upper right')

# Ajustar los márgenes
plt.tight_layout()

# Mostrar el gráfico
plt.show()

#### **Análisis de Correlación**

Podemos analizar la relación entre las diferentes variables

In [ ]:

# Seleccionar variables numéricas relevantes
numeric_vars = ['positive', 'negative', 'death', 'hospitalized', 'totalTestResults']

# Crear matriz de correlación
corr_matrix = df[numeric_vars].corr()

# Crear una máscara para ocultar la mitad inferior de la matriz, pero dejando visible la diagonal
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)

# Crear mapa de calor con anotaciones
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)

plt.title('Matriz de correlación de variables de COVID-19 (Mitad Superior con Diagonal)')
plt.show()
